In [2]:
pip install torch torchvision torchaudio

ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch
Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch 
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler



df_tracks = pd.read_csv("high_popularity_spotify_data.csv")
#df_tracks = pd.read_csv("low_popularity_spotify_data.csv", index_col = 0)

# Load both datasets

#features are energy, tempo, danceability, loudness, liveness, valence, time_signature, speechiness

predictor_features = ["energy", "tempo", "danceability", "loudness",
                     "liveness", "valence",  "time_signature",
                    "speechiness"]

# TO DO - split the data into the predictor features and the target variable (genre)
X = df_tracks[predictor_features]
y = df_tracks["playlist_genre"]

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Convert from pandas DataFrame/Series to tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test, dtype=torch.float32)


label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

X_train = X_train.unsqueeze(1)
X_test = X_test.unsqueeze(1)

device = torch.device("cude" if torch.cuda.is_available() else "cpu")
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)




#Now that data is split into training and test we want to use a Vanilla RNN model in order to train 
class VanillaRNN(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_classes=10):
        super(VanillaRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        #batch_first=True means input tensors are shaped like (batch_size, sequence_length, input_size)
        self.fc = nn.Linear(hidden_size, num_classes)
        #self.fc means it's a fully connected layer that takes the last hidden state and maps it to our 10 output classes

    def forward(self, x):
        h0 = torch.zeros(1, x.shape[0], self.hidden_size, device=x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])  # last time step output
        return out

num_classes = len(np.unique(y_train.cpu().numpy()))
input_size = X_train.shape[2]

model = VanillaRNN(input_size=input_size, hidden_size=64, num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ----- Training Loop -----
epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

# ----- Evaluation -----
model.eval()
with torch.no_grad():
    preds = model(X_test)
    predicted_classes = preds.argmax(dim=1)
    acc = (predicted_classes == y_test).float().mean()
    print(f"\n✅ Test Accuracy: {acc.item()*100:.2f}%")

# Optional: decode genre labels
predicted_genres = label_encoder.inverse_transform(predicted_classes.numpy())
  


Epoch [5/50], Loss: 3.2899
Epoch [10/50], Loss: 3.2251
Epoch [15/50], Loss: 3.1600
Epoch [20/50], Loss: 3.0934
Epoch [25/50], Loss: 3.0246
Epoch [30/50], Loss: 2.9530
Epoch [35/50], Loss: 2.8788
Epoch [40/50], Loss: 2.8025
Epoch [45/50], Loss: 2.7253
Epoch [50/50], Loss: 2.6487

✅ Test Accuracy: 31.36%
